In [1]:
import pandas as pd
import numpy as np


C:\Users\katon\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Read Data

In [2]:
path = "C:/Users/katon/Documents/JHU/DataVisualization/final_project/data/final_data/"

# independent variables
population_df = pd.read_excel(path+"population.xlsx").set_index('State')
housing_df = pd.read_excel(path+"housing.xlsx").set_index('State')

# dependent variables
homeless_df = pd.read_excel(path+"homeless.xlsx").set_index('State')
hpi_df = pd.read_excel(path+"hpi.xlsx").set_index('State')



### Preprocessing

In [3]:
def common_indices(dataframes):
    common_idx = set(dataframes[0].index)
    for df in dataframes[1:]:
        common_idx = common_idx.intersection(df.index)
    return list(common_idx)

In [4]:
common_index = common_indices([homeless_df, housing_df, hpi_df, population_df])

In [5]:
housing_list, population_list, homeless_list, hpi_list, state_list, year_list = [], [], [], [], [], []
for state in common_index:
    for year in list(housing_df.columns):
        housing_list.append(housing_df.loc[state, year])
        population_list.append(population_df.loc[state, year])
        homeless_list.append(homeless_df.loc[state, year])
        hpi_list.append(hpi_df.loc[state, year])
        state_list.append(state)
        year_list.append(year-2000)

In [6]:
homeless_input = pd.DataFrame([housing_list, population_list, hpi_list, state_list, year_list, homeless_list]).T
homeless_input.columns = ['housing', 'population', 'hpi', 'state', 'year', 'homeless']

hpi_input = pd.DataFrame([housing_list, population_list, state_list, year_list, hpi_list]).T
hpi_input.columns = ['housing', 'population', 'state', 'year', 'hpi']

In [7]:
# Prep hpi
hpi_ohe = pd.get_dummies(hpi_input['state'])
hpi_input_df = pd.concat([hpi_input[['housing', 'population', 'year', 'hpi']], hpi_ohe], axis=1)
hpi_input_df = hpi_input_df.astype(int)
hpiX, hpiY = hpi_input_df.drop('hpi', axis=1), hpi_input_df['hpi']


In [8]:
# prep homeless
homeless_ohe = pd.get_dummies(homeless_input['state'])
homeless_input_df = pd.concat([homeless_input[['housing', 'population', 'hpi', 'year', 'homeless']], homeless_ohe], axis=1)
homeless_input_df = homeless_input_df.astype(int)
homelessX, homelessY = homeless_input_df.drop('homeless', axis=1), homeless_input_df['homeless']

## Model Selection

Start with prediction model for HPI, use HPI prediction as feature for homeless prediction

In [9]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score


C:\Users\katon\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


### Building a neural network

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
# HPI
hpi_XTrain, hpi_XTest, hpi_yTrain, hpi_yTest = train_test_split(hpiX, hpiY, test_size=0.2, random_state=42)

# Scale the input data using StandardScaler
hpi_scaler = MinMaxScaler()
hpi_XTrain_scaled = hpi_scaler.fit_transform(hpi_XTrain)
hpi_XTest_scaled = hpi_scaler.transform(hpi_XTest)

# Define the model architecture
hpi_model = keras.Sequential([
    layers.Dense(300, activation='relu'),
    #layers.Dropout(0.1),
    layers.Dense(200, activation='relu'),
    #layers.Dropout(0.1),
    layers.Dense(100, activation='relu'),
    layers.Dense(1)
])

hpi_model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

# Train the model
hpi_model.fit(hpi_XTrain_scaled, hpi_yTrain, epochs=100, batch_size=32, validation_data=(hpi_XTest_scaled, hpi_yTest))

# Evaluate the model on the validation data
val_loss = hpi_model.evaluate(hpi_XTest_scaled, hpi_yTest)

Epoch 1/100
16/16 [==============================] - 1s 54ms/step - loss: 354413.6048 - val_loss: 361402.3438
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 342800.6544 - val_loss: 339109.4062
Epoch 3/100
16/16 [==============================] - 0s 4ms/step - loss: 290086.6967 - val_loss: 251961.1406
Epoch 4/100
16/16 [==============================] - 0s 4ms/step - loss: 196412.0901 - val_loss: 75645.9062
Epoch 5/100
16/16 [==============================] - 0s 4ms/step - loss: 57850.5742 - val_loss: 41818.0898
Epoch 6/100
16/16 [==============================] - 0s 4ms/step - loss: 40670.6739 - val_loss: 26310.5195
Epoch 7/100
16/16 [==============================] - 0s 4ms/step - loss: 23330.6301 - val_loss: 19445.4883
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 15316.7480 - val_loss: 14894.8799
Epoch 9/100
16/16 [==============================] - 0s 4ms/step - loss: 11772.4378 - val_loss: 11827.0303
Epoch 10/100
16/16 [=========

16/16 [==============================] - 0s 4ms/step - loss: 1492.4329 - val_loss: 1946.1317
Epoch 79/100
16/16 [==============================] - 0s 4ms/step - loss: 1505.8091 - val_loss: 1954.4108
Epoch 80/100
16/16 [==============================] - 0s 4ms/step - loss: 1429.5669 - val_loss: 2035.2355
Epoch 81/100
16/16 [==============================] - 0s 4ms/step - loss: 1113.3319 - val_loss: 1884.2444
Epoch 82/100
16/16 [==============================] - 0s 4ms/step - loss: 1384.3994 - val_loss: 1866.8556
Epoch 83/100
16/16 [==============================] - 0s 4ms/step - loss: 1017.4694 - val_loss: 1795.1647
Epoch 84/100
16/16 [==============================] - 0s 4ms/step - loss: 1159.0536 - val_loss: 1759.2281
Epoch 85/100
16/16 [==============================] - 0s 4ms/step - loss: 1152.6598 - val_loss: 1689.8540
Epoch 86/100
16/16 [==============================] - 0s 4ms/step - loss: 985.1180 - val_loss: 1674.4677
Epoch 87/100
16/16 [==============================] - 0s 4ms

In [13]:
# Homeless
homeless_XTrain, homeless_XTest, homeless_yTrain, homeless_yTest = train_test_split(homelessX, homelessY, test_size=0.2, random_state=42)

# Scale the input data using StandardScaler
homeless_scaler = MinMaxScaler()
homeless_XTrain_scaled = homeless_scaler.fit_transform(homeless_XTrain)
homeless_XTest_scaled = homeless_scaler.transform(homeless_XTest)

# Define the model architecture
homeless_model = keras.Sequential([
    layers.Dense(300, activation='relu'),
    #layers.Dropout(0.1),
    layers.Dense(200, activation='relu'),
    #layers.Dropout(0.1),
    layers.Dense(100, activation='relu'),
    layers.Dense(1)
])

homeless_model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

# Train the model
homeless_model.fit(homeless_XTrain_scaled, homeless_yTrain, epochs=100, batch_size=32, validation_data=(homeless_XTest_scaled, homeless_yTest))

# Evaluate the model on the validation data
val_loss = homeless_model.evaluate(homeless_XTest_scaled, homeless_yTest)

Epoch 1/100
16/16 [==============================] - 1s 11ms/step - loss: 524955774.1176 - val_loss: 1079089152.0000
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 407597844.7059 - val_loss: 1078606976.0000
Epoch 3/100
16/16 [==============================] - 0s 4ms/step - loss: 557931169.8824 - val_loss: 1076314240.0000
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 449669630.1176 - val_loss: 1068506752.0000
Epoch 5/100
16/16 [==============================] - 0s 4ms/step - loss: 404941584.0000 - val_loss: 1048039744.0000
Epoch 6/100
16/16 [==============================] - 0s 4ms/step - loss: 359817275.2941 - val_loss: 1004405248.0000
Epoch 7/100
16/16 [==============================] - 0s 4ms/step - loss: 587573304.4706 - val_loss: 933502848.0000
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 410587574.5882 - val_loss: 851863552.0000
Epoch 9/100
16/16 [==============================] - 0s 4ms/step - loss: 

16/16 [==============================] - 0s 4ms/step - loss: 11521590.7647 - val_loss: 27944004.0000
Epoch 73/100
16/16 [==============================] - 0s 5ms/step - loss: 10807682.0294 - val_loss: 26461298.0000
Epoch 74/100
16/16 [==============================] - 0s 4ms/step - loss: 8846352.0588 - val_loss: 26434188.0000
Epoch 75/100
16/16 [==============================] - 0s 4ms/step - loss: 8647992.1176 - val_loss: 26934684.0000
Epoch 76/100
16/16 [==============================] - 0s 4ms/step - loss: 7184489.2941 - val_loss: 24396652.0000
Epoch 77/100
16/16 [==============================] - 0s 4ms/step - loss: 7349652.8088 - val_loss: 26632276.0000
Epoch 78/100
16/16 [==============================] - 0s 4ms/step - loss: 6025423.6912 - val_loss: 25803696.0000
Epoch 79/100
16/16 [==============================] - 0s 4ms/step - loss: 10243627.8235 - val_loss: 24784630.0000
Epoch 80/100
16/16 [==============================] - 0s 4ms/step - loss: 8936561.1471 - val_loss: 2505241

In [14]:
hpi_input.state.unique()

array(['NE', 'AK', 'RI', 'SD', 'MT', 'MD', 'NV', 'ID', 'MA', 'OH', 'CT',
       'UT', 'VT', 'AL', 'OR', 'VA', 'GA', 'CO', 'WV', 'NM', 'ND', 'FL',
       'CA', 'DC', 'AR', 'WY', 'AZ', 'DE', 'KS', 'NC', 'IN', 'MN', 'MO',
       'ME', 'TN', 'NY', 'WI', 'IA', 'IL', 'NH', 'HI', 'MS', 'OK', 'KY',
       'LA', 'TX', 'WA', 'NJ', 'SC', 'PA', 'MI'], dtype=object)

### Making Predictions

The purpose of these models is to make predictions of future HPI and homelessness numbers based on a user inputting the number of houses constructed per year. In order to have this input be done within a Tableau dashboard, the predictions will be premade. This way, the model with not be required each time to make predictions, the dashboard will simply pull from a dataframe

In [15]:
housing_construction = housing_df.copy()
for year in housing_construction.columns[1:]:
    housing_construction[year] = housing_df[year] - housing_df[year-1]
mean_units_per_year = np.mean(housing_construction.iloc[:, 1:], axis=1).astype(int)

In [16]:
def post_scale_hpi(hpi_preds, val_23_100):
    diff = val_23_100 - hpi_preds.loc[100, 2023][0][0] 
    for pct in hpi_preds.index:
        for year in hpi_preds.columns:
            hpi_preds.loc[pct, year] = hpi_preds.loc[pct, year][0][0] + diff
    return hpi_preds

def post_scale_homeless(homeless_preds, val_23_100):
    diff = val_23_100 - homeless_preds.loc[100, 2023][0][0] 
    for pct in homeless_preds.index:
        for year in homeless_preds.columns:
            homeless_preds.loc[pct, year] = homeless_preds.loc[pct, year][0][0] + diff
    return homeless_preds

In [17]:
HPI_dfs = {}
HOMELESS_dfs = {}

for state in hpi_input.state.unique():

    percentages = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]
    years = list(range(2023, 2040))

    mean_state_units = mean_units_per_year[state]

    state_hpi_df = pd.DataFrame(index=percentages, columns=years)
    state_homeless_df = pd.DataFrame(index=percentages, columns=years)

    for year in state_hpi_df.columns:
        for percent in state_hpi_df.index:
            housing_val = int(housing_df.loc[state, 2021] + ((year-2021) * ((percent/100) * mean_units_per_year[state]))) # housing var
            population_val = population_df.loc[state, year] # population var 

            # Predict HPI value
            hpi_in = pd.DataFrame(columns=hpiX.columns)
            hpi_in.loc[0, :] = 0
            hpi_in.loc[0, 'housing'] = housing_val
            hpi_in.loc[0, 'population'] = population_val
            hpi_in.loc[0, 'year'] = year-2000
            hpi_in.loc[0, state] = 1
            # make prediction
            hpi_in = hpi_scaler.transform(hpi_in)
            hpi_prediction = hpi_model.predict(hpi_in)
            state_hpi_df.loc[percent, year] = hpi_prediction

    # Post scale predictions
    diff_21_22 = hpi_df.loc[state, 2022] - hpi_df.loc[state, 2021]
    hpi_val_23_100 = hpi_df.loc[state, 2022] + diff_21_22
    state_hpi_df = post_scale_hpi(state_hpi_df, hpi_val_23_100)


    for year in state_hpi_df.columns:
        for percent in state_hpi_df.index:
            housing_val = int(housing_df.loc[state, 2021] + ((year-2021) * ((percent/100) * mean_units_per_year[state]))) # housing var
            population_val = population_df.loc[state, year] # population var 
            # Predict homeless value
            homeless_in = pd.DataFrame(columns=homelessX.columns)
            homeless_in.loc[0,:] = 0
            homeless_in.loc[0, 'housing'] = housing_val
            homeless_in.loc[0, 'population'] = population_val
            homeless_in.loc[0, 'hpi'] = state_hpi_df.loc[percent, year]
            homeless_in.loc[0, 'year'] = year-2000
            homeless_in.loc[0, state] = 1
            homeless_in = homeless_scaler.transform(homeless_in)
            # make prediction
            homeless_prediction = homeless_model.predict(homeless_in)
            state_homeless_df.loc[percent, year] = homeless_prediction

    # Post scale predictions
    diff_21_22 = homeless_df.loc[state, 2022] - homeless_df.loc[state, 2021]
    homeless_val_23_100 = homeless_df.loc[state, 2022] + diff_21_22
    state_homeless_df = post_scale_homeless(state_homeless_df, homeless_val_23_100)
    
    # Add to dict
    HPI_dfs[state] = state_hpi_df
    HOMELESS_dfs[state] = state_homeless_df


### Reshape into dfs with 2-level header

In [18]:
# Reshape HPI
hpi_list = []
for key, df in HPI_dfs.items():
    hpi_list.append(df)
hpi_df_final = pd.concat(hpi_list, axis=1, keys=HPI_dfs.keys())

# Reshape Homeless
homeless_list = []
for key, df in HOMELESS_dfs.items():
    homeless_list.append(df)
homeless_df_final = pd.concat(homeless_list, axis=1, keys=HOMELESS_dfs.keys())

In [19]:
hpi_df_final

NE                                                              \
           2023        2024        2025        2026        2027        2028   
25   729.009817  782.642263   836.57366  890.602468  944.949697  999.310415   
50   729.009878  782.642507   836.57427  890.603262  944.949636  999.310476   
75   729.009817  782.642751  836.574819  890.604116  944.949331  999.310415   
100      729.01   782.64269   836.57543   890.60491  944.949148  999.310232   
125      729.01  782.642812  836.576101  890.605703  944.948965  999.310171   
150  729.010061  782.642996  836.576772  890.606375  944.948599   999.31011   
175  729.010122  782.643118  836.577444  890.607107  944.948538  999.310049   
200  729.010183   782.64324  836.577993  890.608145  944.948293  999.309927   
225  729.010305   782.64324  836.578848  890.608938  944.948049  999.309805   
250  729.010427  782.643484  836.579458  890.609731  944.947805  999.309683   
275  729.010427  782.643606  836.579885  890.610464  944.947561  999.309683   
300  729.010549  782.643728  836.580557  890.611135  944.947378    999.3095   

                                                         ...           MI  \
            2029         2030         2031         2032  ...         2030   
25   1053.734426  1108.194265  1162.680105   1217.17156  ...  1095.957556   
50   1053.733328  1108.193044  1162.678884  1217.169851  ...  1095.956091   
75   1053.732351  1108.191946  1162.677175  1217.168142  ...  1095.954749   
100   1053.73113  1108.190603  1162.675588  1217.166311  ...   1095.95304   
125  1053.730276  1108.189749  1162.674124  1217.164724  ...  1095.951697   
150  1053.729299  1108.188528   1162.67217  1217.162771  ...  1095.950476   
175  1053.728079  1108.187307  1162.670706   1217.16094  ...  1095.948645   
200  1053.727102  1108.185964  1162.668997  1217.159109  ...  1095.947424   
225  1053.726125  1108.184866  1162.667532  1217.157644  ...  1095.945959   
250  1053.725149  1108.183767  1162.665945  1217.155691  ...  1095.944495   
275  1053.724172  1108.182546  1162.664114  1217.153982  ...  1095.942908   
300  1053.723196   1108.18157  1162.662649  1217.152151  ...  1095.941443   

                                                                      \
            2031         2032         2033         2034         2035   
25   1150.332556   1204.76554  1259.225012  1313.688391  1368.168738   
50   1150.331091   1204.76261  1259.221838   1313.68656  1368.167029   
75   1150.329504  1204.759558   1259.21842  1313.684363  1368.164709   
100  1150.327917   1204.75614  1259.215002  1313.682654  1368.162878   
125  1150.326331   1204.75321  1259.211584  1313.680701  1368.160559   
150  1150.324622  1204.750159  1259.208044  1313.678992  1368.158484   
175  1150.322913  1204.746741  1259.204626   1313.67655  1368.156531   
200  1150.321448  1204.743933  1259.201331  1313.674963  1368.154578   
225  1150.319617  1204.741003  1259.198035   1313.67301  1368.152502   
250  1150.318274  1204.737585  1259.194617  1313.670813  1368.150183   
275  1150.316565  1204.734656  1259.191199  1313.669226  1368.147986   
300  1150.314978  1204.731604  1259.188391  1313.667151  1368.145911   

                                                         
            2036         2037         2038         2039  
25   1422.655432  1477.160315  1531.665564  1586.170691  
50   1422.653845  1477.159216  1531.663977  1586.168982  
75   1422.652747  1477.157629  1531.662634  1586.167639  
100  1422.651648  1477.156531  1531.661169  1586.166174  
125  1422.650305  1477.155066  1531.660071  1586.164587  
150  1422.648962  1477.153601   1531.65824  1586.163123  
175  1422.647864  1477.152258  1531.656653   1586.16178  
200  1422.646399   1477.15116  1531.655432  1586.159949  
225  1422.644934  1477.149817  1531.654089  1586.158728  
250   1422.64408   1477.14823  1531.652625  1586.156897  
275  1422.642249  1477.146643   1531.65116  1586.155432  
300  1422.641272  1477.145667  1531.649817  1586.154089  

[12 r

In [20]:
# Write to Excel
hpi_df_final.to_excel(path+'final_hpi_projections.xlsx')
homeless_df_final.to_excel(path+'final_homeless_projections.xlsx')

<ipython-input-20-1a5996b94719>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  hpi_df_final.to_excel(path+'final_hpi_projections.xlsx')
<ipython-input-20-1a5996b94719>:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  homeless_df_final.to_excel(path+'final_homeless_projections.xlsx')
